In [ ]:
!pip install -q einops
!pip install cohere
!pip install accelerate
!pip install bitsandbytes

#import
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, GenerationConfig, TextStreamer, LlamaForCausalLM, LlamaTokenizer

import torch
from inspect import cleandoc
import cohere

In [ ]:
#function
#TODO AGGIUNGERE PLOT E DIFFERENZE + REFACTORING
def trova_elemento_piu_comune(array_di_coppie):
    # Unisci tutte le coppie in un unico elenco
    tutti_elementi = [elemento for coppia in array_di_coppie for elemento in coppia]

    # Conta le occorrenze di ciascun elemento
    conteggio_elementi = Counter(tutti_elementi)

    # Trova l'elemento più comune
    elemento_piu_comune, conteggio_max = conteggio_elementi.most_common(1)[0]

    return elemento_piu_comune, conteggio_max

def elimina_coppie_con_elemento(array_di_coppie, elemento_da_elim):
    # Utilizza una list comprehension per filtrare le coppie che non contengono l'elemento da eliminare
    nuove_coppie = [coppia for coppia in array_di_coppie if elemento_da_elim not in coppia]

    return nuove_coppie

def elimina_elemento_coppie(indices2,to_remove):
    for pair in indices2:
        to_remove.append(pair[0])
    indices2.clear()

In [ ]:
#lettura dataset originale + aggiunta id
df = pd.read_csv('/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv')
df['id'] = range(0, len(df))
df

In [ ]:
# conteggio label dataset
label_counts = df['label'].value_counts()

plt.figure(figsize=(8, 6))
sns.barplot(x=label_counts.index, y=label_counts.values)
plt.title('Hist')
plt.xlabel('Generated')
plt.ylabel('Count')
plt.show()

print(label_counts)

In [ ]:
# per ogni prompt contiamo le label
prompt_count = df.groupby(['prompt_name', 'label']).size().unstack()
prompt_count

prompt_df = pd.DataFrame(columns=['prompt_name', 'human', 'ai'])
grouped_df = df.groupby('prompt_name')['label'].value_counts().unstack().reset_index()

# Assegnare i risultati al nuovo DataFrame
prompt_df['prompt_name'] = grouped_df['prompt_name']
prompt_df['human'] = grouped_df[0]
prompt_df['ai'] = grouped_df[1]

prompt_df

In [ ]:
# Similarità tra i testi umani e individuazione id da eliminare

ids_to_remove = []

#qui ci stanno i prompt che contengono similarità tra i testi umani
#prompt_similarity = ['Car-free cities','Does the electoral college work?','Phones and driving']

for prompt in prompt_df['prompt_name'].unique():
    # Filter the dataframe based on label and prompt_name
    filtered_df = df.loc[(df['label'] == 0) & (df['prompt_name'] == prompt), ['text', 'id']]

    # Create a TfidfVectorizer
    vectorizer = TfidfVectorizer()

    # Fit and transform the text data
    tfidf_matrix = vectorizer.fit_transform(filtered_df['text'])

    # Calculate cosine similarity matrix
    cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

    # Create a DataFrame with cosine similarity values
    cosine_sim_df = pd.DataFrame(cosine_sim_matrix, columns=filtered_df['id'], index=filtered_df['id'])

    # Set a threshold
    threshold = 0.9

    # Find indices of pairs with similarity greater than the threshold
    indices = [(filtered_df.index[i], filtered_df.index[j]) for i in range(len(cosine_sim_df))
               for j in range(i + 1, len(cosine_sim_df.columns))
               if cosine_sim_df.iloc[i, j] > threshold]
    
    while len(indices)>0:
        da_eliminare, count = trova_elemento_piu_comune(indices)
        if count > 1:
            indices = elimina_coppie_con_elemento(indices,da_eliminare)
            ids_to_remove.append(da_eliminare)
        else:
            elimina_elemento_coppie(indices,ids_to_remove)

In [ ]:
# eliminiamo gli id umani con alta similarità
df = df[~df['id'].isin(ids_to_remove)]

In [ ]:
# conteggio label dataset ripulito
label_counts = df['label'].value_counts()

plt.figure(figsize=(8, 6))
sns.barplot(x=label_counts.index, y=label_counts.values)
plt.title('Hist')
plt.xlabel('Generated')
plt.ylabel('Count')
plt.show()

print(label_counts)

In [ ]:
# per ogni prompt contiamo le label dal dataset ripulito
prompt_count = df.groupby(['prompt_name', 'label']).size().unstack()
prompt_count

prompt_df = pd.DataFrame(columns=['prompt_name', 'human', 'ai'])
grouped_df = df.groupby('prompt_name')['label'].value_counts().unstack().reset_index()

# Assegnare i risultati al nuovo DataFrame
prompt_df['prompt_name'] = grouped_df['prompt_name']
prompt_df['human'] = grouped_df[0]
prompt_df['ai'] = grouped_df[1]

prompt_df

In [ ]:
#TODO Similarità AI

In [ ]:
treshold = 500
# prompt che hanno bisogno di generazione di nuovi testi con label 1
to_generate = prompt_df[prompt_df['ai'] + treshold < prompt_df['human']]
to_generate['difference'] = to_generate['human'] - to_generate['ai']
to_generate

In [ ]:
# controlliamo lunghezza testi (ci servirà per la generazione)
df['text_length'] = df['text'].str.len()
text_len_mean = int(df.query("label == 0")['text_length'].mean())
text_len_std = int(df.query("label == 0")['text_length'].std())

print(f"Mean length of train essays by human in our dataset: {text_len_mean}")
print(f"Mean standard deviation of train essays by human in our dataset: {text_len_std}")

df = df.drop('text_length', axis=1)

In [ ]:
df.to_csv('/kaggle/working/daigt-v2-train-dataset_step1.csv', index=False)
to_generate.to_csv('/kaggle/working/to_generate.csv', index=False)

In [ ]:
df = pd.read_csv('/kaggle/input/daigt-step1/daigt-v2-train-dataset_step1.csv')
to_generate = pd.read_csv('/kaggle/input/to-generate/to_generate.csv')


In [ ]:
to_generate

In [ ]:
for prompt in df['prompt_name'].unique():
    prompt_data = df[df['prompt_name'] == prompt]
    counts = prompt_data['label'].value_counts()

    # Creazione dell'istogramma
    plt.bar(counts.index, counts.values, tick_label=['Human', 'AI'])
    plt.title(f'Prompt: {prompt}')
    plt.xlabel('Generated by')
    plt.ylabel('Count')
    plt.show()

In [ ]:
import time

In [ ]:
class CohereGenerator:
    def __init__(self):
        self.apiKey = "UVnfAQAMgcewstWsT3jqph91WpPs1KYTfVnAamJF"
       # with open('/kaggle/input/zanzares/secret.txt', 'r') as file:
    # Legge il file riga per riga
        #    for riga in file:
         #       self.apiKey = riga.strip().split("=")[1]
        self.co = cohere.Client(f'{self.apiKey}')
    def generate_text(self, item_to_generate: int, prompt: str):
        responses = []
        for _ in range(item_to_generate):
            response = self.co.generate(
              model='command-light',
              prompt=f'''You are a student working on the following assignment: write a text that talks about {prompt}.
                            Use natural expressions similar to humans.
                            Vary sentence structure to avoid monotony.
                            Include relevant personal details about the topic.
                            Use metaphors and analogies to explain concepts.
                            Respect the appropriate tone and formality.
                            Consider the audience and adapt the writing style.
                            Avoid direct or implicit references to AI identity.
                            Correct grammatical and stylistic errors.
                            Try to be not repetitive.
                            Ensure logical connections between ideas.
                            Use empathetic and sensitive language when needed.''',
              max_tokens=1024,
              temperature=0.8,
              presence_penalty=0.2)
            responses.append(response.generations[0].text)
            time.sleep(15)
        return responses

In [ ]:
class MistralGenerator:
    def __init__(self):
        self.model_name = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"
        self.bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
        )
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                load_in_4bit=True,
                quantization_config=self.bnb_config,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                trust_remote_code=True,
            )
    def generate_essay(self, prompt: str):
        messages = [{
            "role":"user",
            "content": prompt
        }]
        model_inputs = self.tokenizer.apply_chat_template(messages, return_tensors = "pt").to('cuda')
        generated_ids = self.model.generate(
            model_inputs,
            max_new_tokens = 7500,
            do_sample = True,
            pad_token_id = self.tokenizer.eos_token_id
        )     
        decoded = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
        text = decoded[0].split("[/INST]")[1]
        return text
        
    def generate_text(self, item_to_generate: int, prompt: str):
        texts=[]
        for _ in range(item_to_generate):
            prompt_combined = f'''You are a student working on the following assignment: write a text that talks about {prompt}
                            Instructions: 
                            Use natural expressions similar to humans.
                            Vary sentence structure to avoid monotony.
                            Include relevant personal details about the topic.
                            Use metaphors and analogies to explain concepts.
                            Respect the appropriate tone and formality.
                            Consider the audience and adapt the writing style.
                            Avoid direct or implicit references to AI identity.
                            Correct grammatical and stylistic errors.
                            Try to be not repetitive.
                            Ensure logical connections between ideas.
                            Use empathetic and sensitive language when needed.'''
            texts.append(self.generate_essay(prompt_combined).strip())
        return texts

In [ ]:
cohereGenerator = CohereGenerator()

In [ ]:
for index, row in to_generate.iterrows():
    if(row['difference']> 1000):
        prompt_name=row['prompt_name']
        itemsForGenerator = row['difference']//4
        texts = cohereGenerator.generate_text(itemsForGenerator, prompt_name)
        data = {'text': texts,
            'label': [1] * len(texts),
            'prompt_name': [prompt_name] * len(texts),
            'source': ['cohere'] * len(texts)}
        cohere_df = pd.DataFrame(data)
        cohere_df.to_csv(f'/kaggle/working/cohere_{prompt_name}.csv', index=False)
        df = pd.concat([df,cohere_df], ignore_index=True)

In [ ]:
df.to_csv('/kaggle/working/daigt-v2-train-dataset_step2_cohere.csv', index=False)

In [ ]:
mistralGenerator = MistralGenerator()

In [ ]:
for index, row in to_generate.iterrows():
    if(row['difference']> 1000):
        prompt_name=row['prompt_name']
        itemsForGenerator = row['difference']//4
        texts = mistralGenerator.generate_text(itemsForGenerator, prompt_name)
        data = {'text': texts,
            'label': [1] * len(texts),
            'prompt_name': [prompt_name] * len(texts),
            'source': ['mistral'] * len(texts)}
        mistral_df = pd.DataFrame(data)
        mistral_df.to_csv(f'/kaggle/working/mistral_{prompt_name}.csv', index=False)
        df = pd.concat([df,mistral_df], ignore_index=True)

In [ ]:
df.to_csv('/kaggle/working/daigt-v3.csv', index=False)